# Flood Segmentation with PyTorch U-Net (Sentinel-1 SAR Flood Dataset)

## Introduction

Flood mapping from satellite imagery is a crucial task for disaster response. In this notebook, we replicate a PyTorch U-Net model for segmenting floodwater in Sentinel-1 SAR images, based on the Cloud to Street – Microsoft Flood Dataset. The U-Net architecture is a convolutional neural network designed for pixel-wise segmentation tasks. Our goal is to download and prepare the dataset, define the U-Net model, train it on the SAR images with corresponding water masks, evaluate the segmentation performance (e.g. using Intersection over Union), and visualize some prediction results.

Example Sentinel-1 SAR image (left) of a flood event and its corresponding ground-truth water mask (right). The SAR image shows flood-affected areas in radar intensity, and the mask highlights water pixels in white.

## Installing Dependencies

First, we install and import the necessary packages. We use PyTorch for building and training the U-Net, rasterio for reading geospatial image files (GeoTIFF format), and other common libraries for data handling and visualization.

In [1]:
!pip install torch torchvision rasterio numpy pandas matplotlib tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 41.4 MB/s eta 0:00:00


In [2]:
# Import required libraries
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import rasterio
from tqdm import tqdm
import random

# Check PyTorch version and availability of GPU
print("PyTorch version:", torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

PyTorch version: 2.8.0+cu126
Using device: cpu


## Dataset Download and Preparation

### Dataset Overview

The Cloud to Street – Microsoft Flood dataset contains Sentinel-1 SAR image chips (with VV and VH polarizations) and corresponding binary flood masks. Each image chip covers an area affected by a flood event, and the mask indicates flooded vs. non-flooded pixels (permanent water and floodwater).

### Download Instructions

The dataset can be downloaded from Radiant Earth MLHub. You'll need to download two files:
- `c2smsfloods_v1_source_s1.tar.gz` (Sentinel-1 VV/VH images)
- `c2smsfloods_v1_labels_s1_water.tar.gz` (corresponding water masks)

You can download them using AWS CLI:
```bash
aws s3 cp --no-sign-request s3://radiant-mlhub/c2smsfloods/c2smsfloods_v1_source_s1.tar.gz .
aws s3 cp --no-sign-request s3://radiant-mlhub/c2smsfloods/c2smsfloods_v1_labels_s1_water.tar.gz .
```

Or use the Radiant MLHub Python API if you have an API key.

### Extraction and Tiling

We'll extract the downloaded files and tile each 512×512 chip into four 256×256 images to increase training samples.

In [ ]:
import tarfile

# Paths to the downloaded tar files (update these if needed)
s1_tar_path = "c2smsfloods_v1_source_s1.tar.gz"
label_tar_path = "c2smsfloods_v1_labels_s1_water.tar.gz"

# Extract tar files to temporary directories
extract_dir_s1 = Path("source_data")
extract_dir_labels = Path("label_data")
extract_dir_s1.mkdir(parents=True, exist_ok=True)
extract_dir_labels.mkdir(parents=True, exist_ok=True)

print("Extracting Sentinel-1 source data...")
with tarfile.open(s1_tar_path, 'r:gz') as tar:
    tar.extractall(path=extract_dir_s1)
print("Extracting water label data...")
with tarfile.open(label_tar_path, 'r:gz') as tar:
    tar.extractall(path=extract_dir_labels)

# After extraction, we expect 'train_features' and 'train_labels' directories
feature_dir = extract_dir_s1 / "train_features"
label_dir   = extract_dir_labels / "train_labels"
assert feature_dir.exists() and label_dir.exists(), "Extracted directories not found. Check extraction paths."

# Create output directories for 256x256 tiles
tile_dir_vv = Path("data/chips/VV"); tile_dir_vv.mkdir(parents=True, exist_ok=True)
tile_dir_vh = Path("data/chips/VH"); tile_dir_vh.mkdir(parents=True, exist_ok=True)
tile_dir_label = Path("data/labels"); tile_dir_label.mkdir(parents=True, exist_ok=True)

# Tiling each chip
tile_count = 0
print("Tiling chips into 256x256 patches...")

# Get all label files
label_files = sorted(label_dir.glob("*.tif"))

for label_path in tqdm(label_files):
    chip_id = label_path.stem  # e.g. "awc00" from "awc00.tif"
    vv_path = feature_dir / f"{chip_id}_vv.tif"
    vh_path = feature_dir / f"{chip_id}_vh.tif"
    if not vv_path.exists() or not vh_path.exists():
        continue  # skip if corresponding images not found

    # Read label and polarization images
    with rasterio.open(label_path) as lab_src, \
         rasterio.open(vv_path) as vv_src, \
         rasterio.open(vh_path) as vh_src:
        label_img = lab_src.read(1)           # 2D array
        vv_img = vv_src.read(1)              # 2D array
        vh_img = vh_src.read(1)              # 2D array

        # Replace no-data values (-1) in label with 0 (non-water)
        if lab_src.nodata is not None:
            nodata_val = lab_src.nodata
            label_img[label_img == nodata_val] = 0

        H, W = label_img.shape
        tile_size = 256
        # Slide a 256x256 window over the image (non-overlapping)
        for y in range(0, H, tile_size):
            for x in range(0, W, tile_size):
                # Ensure we don't go out of bounds
                if y+tile_size > H or x+tile_size > W:
                    continue
                sub_lab = label_img[y:y+tile_size, x:x+tile_size]
                sub_vv  = vv_img[y:y+tile_size, x:x+tile_size]
                sub_vh  = vh_img[y:y+tile_size, x:x+tile_size]

                # Define tile file name based on chip and tile indices
                tile_id = f"{chip_id}_{int(y/tile_size)}_{int(x/tile_size)}"

                # Save the tile images using rasterio
                # Save VV
                profile = vv_src.profile.copy()
                profile.update(height=tile_size, width=tile_size, count=1, transform=None)
                with rasterio.open(tile_dir_vv / f"{tile_id}.tif", 'w', **profile) as dst:
                    dst.write(sub_vv, 1)
                # Save VH
                profile = vh_src.profile.copy()
                profile.update(height=tile_size, width=tile_size, count=1, transform=None)
                with rasterio.open(tile_dir_vh / f"{tile_id}.tif", 'w', **profile) as dst:
                    dst.write(sub_vh, 1)
                # Save label (as uint8)
                profile = lab_src.profile.copy()
                profile.update(height=tile_size, width=tile_size, count=1, dtype=rasterio.uint8, transform=None, nodata=None)
                with rasterio.open(tile_dir_label / f"{tile_id}.tif", 'w', **profile) as dst:
                    dst.write(sub_lab.astype(np.uint8), 1)

                tile_count += 1

print(f"Tiling completed: {tile_count} tiles created.")

## Dataset Preprocessing and Loading

Now we'll create a custom PyTorch Dataset to load the tiled data, normalize the SAR images, and split the data into training and test sets.

In [ ]:
# Define custom Dataset
class FloodDataset(Dataset):
    def __init__(self, ids, data_dir_vv, data_dir_vh, data_dir_label, transform=None):
        """
        ids: list of tile IDs (filenames without extension)
        data_dir_vv, data_dir_vh, data_dir_label: directories containing VV, VH, and label tiles
        transform: optional augmentation or preprocessing function
        """
        self.ids = ids
        self.data_dir_vv = data_dir_vv
        self.data_dir_vh = data_dir_vh
        self.data_dir_label = data_dir_label
        self.transform = transform

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        tile_id = self.ids[idx]
        vv_path = self.data_dir_vv / f"{tile_id}.tif"
        vh_path = self.data_dir_vh / f"{tile_id}.tif"
        label_path = self.data_dir_label / f"{tile_id}.tif"

        # Read images using rasterio
        with rasterio.open(vv_path) as vv_src:
            vv = vv_src.read(1).astype(np.float32)
        with rasterio.open(vh_path) as vh_src:
            vh = vh_src.read(1).astype(np.float32)
        with rasterio.open(label_path) as lab_src:
            mask = lab_src.read(1).astype(np.uint8)

        # Normalize VV and VH to [0,1] range (min-max normalization per tile)
        if vv.max() > vv.min():  # avoid division by zero
            vv = (vv - vv.min()) / (vv.max() - vv.min())
        else:
            vv = vv * 0.0
        if vh.max() > vh.min():
            vh = (vh - vh.min()) / (vh.max() - vh.min())
        else:
            vh = vh * 0.0

        # Expand dims to [C, H, W] and convert to tensor
        vv = torch.from_numpy(vv).unsqueeze(0)  # shape [1,256,256]
        vh = torch.from_numpy(vh).unsqueeze(0)  # shape [1,256,256]
        image = torch.cat([vv, vh], dim=0)      # shape [2,256,256]
        mask = torch.from_numpy(mask).unsqueeze(0).float()  # shape [1,256,256], as float (0/1)

        if self.transform:
            # Apply transforms (e.g., augmentation) - not implemented in this example
            image, mask = self.transform(image, mask)

        return image, mask

# Gather all tile IDs from the data directories
tile_ids = [f.stem for f in tile_dir_label.glob("*.tif")]
tile_ids = sorted(tile_ids)
print(f"Total tiles available: {len(tile_ids)}")

# Shuffle and split into train/test
random.seed(42)
random.shuffle(tile_ids)
split_idx = int(0.8 * len(tile_ids))  # 80% for training
train_ids = tile_ids[:split_idx]
test_ids  = tile_ids[split_idx:]
print(f"Training tiles: {len(train_ids)}, Test tiles: {len(test_ids)}")

# Create Dataset and DataLoader
train_dataset = FloodDataset(train_ids, tile_dir_vv, tile_dir_vh, tile_dir_label)
test_dataset  = FloodDataset(test_ids,  tile_dir_vv, tile_dir_vh, tile_dir_label)

# For training, we'll use a batch size (e.g. 8 or 16 depending on memory)
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Inspect a sample batch shape
images, masks = next(iter(train_loader))
print("Batch image tensor shape:", images.shape)  # (B, 2, 256, 256)
print("Batch mask tensor shape:", masks.shape)    # (B, 1, 256, 256)

## U-Net Model Architecture

The U-Net is a convolutional network that consists of an encoder (downsampling path) and a decoder (upsampling path) with skip connections between corresponding layers.

In [ ]:
# Define the U-Net model components
class DoubleConv(nn.Module):
    """Two consecutive conv (3x3) layers with ReLU (encoder or decoder block)."""
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.net(x)

class Down(nn.Module):
    """Downsampling: MaxPool then DoubleConv."""
    def __init__(self, in_channels, out_channels):
        super(Down, self).__init__()
        self.pool = nn.MaxPool2d(2)
        self.conv = DoubleConv(in_channels, out_channels)
    def forward(self, x):
        x = self.pool(x)
        x = self.conv(x)
        return x

class Up(nn.Module):
    """Upsampling: ConvTranspose (2x2) then DoubleConv. Skip connection from encoder is concatenated."""
    def __init__(self, in_channels, out_channels):
        super(Up, self).__init__()
        # ConvTranspose2d to upsample by 2
        self.up = nn.ConvTranspose2d(in_channels, in_channels//2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)
    def forward(self, x1, x2):
        # x1 is decoder feature, x2 is encoder feature to concatenate
        x1 = self.up(x1)
        # Pad x1 if needed when dimensions are odd
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        if diffY != 0 or diffX != 0:
            # pad the upsampled feature map so that it matches size of encoder feature map
            x1 = F.pad(x1, [0, diffX, 0, diffY])
        # Concatenate along channels
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x

class OutConv(nn.Module):
    """1x1 convolution for output."""
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
    def forward(self, x):
        return self.conv(x)

# Full U-Net Model
class UNet(nn.Module):
    def __init__(self, n_channels=2, n_classes=1):
        super(UNet, self).__init__()
        self.in_conv = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)  # bottom layer
        self.up1   = Up(512, 256)
        self.up2   = Up(256, 128)
        self.up3   = Up(128, 64)
        self.up4   = Up(64, 64)
        self.out_conv = OutConv(64, n_classes)
    def forward(self, x):
        # Encoder (downsampling)
        x1 = self.in_conv(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        # Decoder (upsampling)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        # Output layer
        logits = self.out_conv(x)
        return logits

model = UNet(n_channels=2, n_classes=1).to(device)
print("U-Net model initialized. Number of parameters:", sum(p.numel() for p in model.parameters()))

## Training the Model

We'll use Binary Cross-Entropy loss with logits and the Adam optimizer. During training, we'll track the training loss and evaluate on the test set to monitor the Intersection over Union (IoU) score.

In [ ]:
# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 5  # for demonstration; you can increase this for better performance
train_losses = []
test_losses = []
test_ious = []

print("Starting training...")
for epoch in range(1, num_epochs+1):
    model.train()
    running_loss = 0.0

    # Training loop with progress bar
    train_pbar = tqdm(train_loader, desc=f'Epoch {epoch}/{num_epochs} [Train]')
    for images, masks in train_pbar:
        images = images.to(device)
        masks = masks.to(device)
        optimizer.zero_grad()
        # Forward pass
        logits = model(images)              # shape [B,1,256,256]
        loss = criterion(logits, masks)     # compute loss against ground truth mask
        # Backpropagation
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        train_pbar.set_postfix({'loss': loss.item()})

    epoch_loss = running_loss / len(train_dataset)
    train_losses.append(epoch_loss)

    # Evaluate on test set for this epoch
    model.eval()
    test_loss = 0.0
    # IoU components
    inter_sum = 0
    union_sum = 0

    test_pbar = tqdm(test_loader, desc=f'Epoch {epoch}/{num_epochs} [Test]')
    with torch.no_grad():
        for images, masks in test_pbar:
            images = images.to(device)
            masks = masks.to(device)
            logits = model(images)
            # Compute loss
            loss = criterion(logits, masks)
            test_loss += loss.item() * images.size(0)
            # Compute IoU
            probs = torch.sigmoid(logits)  # convert logits to probabilities
            preds = (probs >= 0.5).float()  # threshold to binary predictions
            # Intersection and Union for IoU
            inter = (preds * masks).sum(dim=[1,2,3])  # sum of element-wise multiplication (true positives per image)
            union = ((preds + masks) >= 1).float().sum(dim=[1,2,3])  # sum of element-wise logical OR
            inter_sum += inter.sum().item()
            union_sum += union.sum().item()

            current_iou = inter_sum / union_sum if union_sum > 0 else 0.0
            test_pbar.set_postfix({'iou': current_iou})

    avg_test_loss = test_loss / len(test_dataset)
    test_losses.append(avg_test_loss)
    # Calculate IoU score
    iou_score = inter_sum / union_sum if union_sum > 0 else 0.0
    test_ious.append(iou_score)
    print(f"Epoch {epoch}/{num_epochs}: Train Loss = {epoch_loss:.4f}, Test Loss = {avg_test_loss:.4f}, Test IoU = {iou_score:.4f}")

## Training Progress Visualization

Let's plot the training curve and IoU to visualize the training progress.

In [ ]:
# Plot training and test loss over epochs
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs+1), train_losses, label="Train Loss")
plt.plot(range(1, num_epochs+1), test_losses, label="Test Loss")
plt.xlabel("Epoch")
plt.ylabel("BCE Loss")
plt.title("Loss Curve")
plt.legend()

# Plot IoU over epochs
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs+1), test_ious, marker='o', color='green')
plt.xlabel("Epoch")
plt.ylabel("IoU")
plt.title("Test IoU over Epochs")
plt.grid(True)

plt.tight_layout()
plt.show()

## Evaluation on Test Set

After training, we evaluate the model on the test set to obtain final metrics.

In [ ]:
model.eval()
inter_sum = 0
union_sum = 0
all_preds = []
all_masks = []

with torch.no_grad():
    for images, masks in tqdm(test_loader, desc="Evaluating"):
        images = images.to(device)
        masks = masks.to(device)
        logits = model(images)
        preds = (torch.sigmoid(logits) >= 0.5).float()
        inter = (preds * masks.to(device)).sum(dim=[1,2,3])
        union = ((preds + masks.to(device)) >= 1).float().sum(dim=[1,2,3])
        inter_sum += inter.sum().item()
        union_sum += union.sum().item()

        # Store for additional metrics if needed
        all_preds.append(preds.cpu())
        all_masks.append(masks.cpu())

iou_final = inter_sum / union_sum if union_sum > 0 else 0.0
print(f"Final Test IoU: {iou_final:.4f}")

# Calculate additional metrics
all_preds = torch.cat(all_preds)
all_masks = torch.cat(all_masks)

# Pixel accuracy
accuracy = (all_preds == all_masks).float().mean()
print(f"Pixel Accuracy: {accuracy:.4f}")

# Precision and Recall
true_positives = (all_preds * all_masks).sum()
false_positives = (all_preds * (1 - all_masks)).sum()
false_negatives = ((1 - all_preds) * all_masks).sum()

precision = true_positives / (true_positives + false_positives + 1e-8)
recall = true_positives / (true_positives + false_negatives + 1e-8)
f1_score = 2 * (precision * recall) / (precision + recall + 1e-8)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")

## Visualization of Segmentation Results

Finally, let's visualize some predictions versus ground truth on the test set.

In [ ]:
# Visualize a few test samples with their predicted masks
model.eval()
num_samples_to_show = 3
samples = random.sample(test_ids, num_samples_to_show)

plt.figure(figsize=(12, num_samples_to_show*4))
for i, tile_id in enumerate(samples, start=1):
    # Load the VV image, VH image, and true mask for the tile
    vv_path = tile_dir_vv / f"{tile_id}.tif"
    vh_path = tile_dir_vh / f"{tile_id}.tif"
    lab_path = tile_dir_label / f"{tile_id}.tif"
    with rasterio.open(vv_path) as vv_src:
        vv = vv_src.read(1).astype(np.float32)
    with rasterio.open(vh_path) as vh_src:
        vh = vh_src.read(1).astype(np.float32)
    with rasterio.open(lab_path) as lab_src:
        true_mask = lab_src.read(1).astype(np.uint8)
        true_mask[true_mask != 1] = 0  # ensure binary

    # Normalize VV for display
    vv_disp = (vv - vv.min()) / (vv.max() - vv.min() + 1e-9)
    # Get model prediction
    img_tensor = torch.from_numpy(np.stack([vv, vh], axis=0)).unsqueeze(0).to(device)  # shape [1,2,H,W]
    with torch.no_grad():
        logit = model(img_tensor)
        pred_prob = torch.sigmoid(logit).cpu().numpy()[0,0]  # probability map
        pred_mask = (pred_prob >= 0.5).astype(np.uint8)  # binary mask array

    # Plot SAR image (VV polarization)
    plt.subplot(num_samples_to_show, 4, 4*(i-1) + 1)
    plt.imshow(vv_disp, cmap='gray')
    plt.title(f"SAR Image (VV)\n{tile_id}")
    plt.axis('off')

    # Plot SAR image (VH polarization)
    plt.subplot(num_samples_to_show, 4, 4*(i-1) + 2)
    vh_disp = (vh - vh.min()) / (vh.max() - vh.min() + 1e-9)
    plt.imshow(vh_disp, cmap='gray')
    plt.title("SAR Image (VH)")
    plt.axis('off')

    # Plot true mask
    plt.subplot(num_samples_to_show, 4, 4*(i-1) + 3)
    plt.imshow(true_mask, cmap='Blues')  # Blues colormap to highlight water
    plt.title("Ground Truth Mask")
    plt.axis('off')

    # Plot predicted mask
    plt.subplot(num_samples_to_show, 4, 4*(i-1) + 4)
    plt.imshow(pred_mask, cmap='Blues')
    plt.title("Predicted Mask")
    plt.axis('off')

plt.tight_layout()
plt.show()

## Conclusion

This notebook demonstrated a complete pipeline for flood segmentation using PyTorch U-Net on Sentinel-1 SAR data. Key steps included:

1. **Data Preparation**: Downloading, extracting, and tiling the Cloud to Street – Microsoft Flood dataset
2. **Model Architecture**: Implementing a U-Net with encoder-decoder structure and skip connections
3. **Training**: Using BCE loss and Adam optimizer to train the segmentation model
4. **Evaluation**: Measuring performance with IoU, precision, recall, and F1-score
5. **Visualization**: Comparing predictions with ground truth masks

The model successfully learned to identify flood areas in SAR imagery. For improved performance, consider:
- Training for more epochs
- Using data augmentation
- Incorporating additional input channels (DEM, permanent water maps)
- Implementing the dual-model approach from the original paper
- Using more sophisticated loss functions (Dice loss, Focal loss)

This approach can be valuable for rapid flood mapping and disaster response applications.